In [30]:
from modules.eve import Eve
import pandas as pd
from modules.models_manager.models_manager import ModelsManager
import networkx as nx
from pyvis.network import Network

In [4]:
eve = Eve()
eve.initialize(username='farzad')

In [136]:
dfg = eve.user_manager.get_keyphrases()
dfg.head(5)

node_id  session_id                  node           parent_node
0        1           1  Minimalist Lifestyle  Personal Development
1        2           1  Minimalist Lifestyle  Personal Development
2        3           1  Minimalist Lifestyle  Personal Development
3        4           1  Minimalist Lifestyle  Personal Development
4        5           1  Minimalist Lifestyle  Personal Development

In [159]:
bad = '"The Home & Living category focuses on aspects of managing and living in a home, from organization and cleaning to decor and maintenance. It provides information and advice on creating a comfortable, functional living environment. This can also encompass topics like gardening, DIY home projects, and sustainable living practices."'
# if bad changed to Home & Living
for row in dfg.index:
    if dfg.loc[row, 'parent_node'] == bad:
        dfg.loc[row, 'parent_node'] = 'Home & Living'

In [174]:
# group by node and parent node
dfgg = dfg.groupby(['node', 'parent_node']).size().reset_index(name='count')

In [176]:
dfgg.head(5)

node           parent_node  count
0         Achievements         Personal Growt     14
1     Anxiety and Fears         Mental Health     15
2  Budgeting and Saving               Finance      7
3    Career Progression                   Wor     14
4     Career and Financ  Personal Development     39

In [183]:
# normalize the count
dfgg['count'] = dfgg['count'] / dfgg['count'].max()

In [184]:
nodes = pd.concat([dfgg['node'], dfgg['parent_node']], axis=0).unique()
nodes.shape

(76,)

In [ ]:

G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfgg.iterrows():
    G.add_edge(
        str(row["node"]),
        str(row["parent_node"]),
        weight=row["count"]
    )

In [186]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  20
[['Achievements ', 'Personal Growt', 'Resilience and Grit'], ['Anxiety and Fears', 'Coping Strategies', 'Dreams (Sleep)', 'Emotions', 'Health and Wellness', 'Mental Health', 'Self-Care'], ['Budgeting and Saving', 'Finance'], ['Career Development', 'Work'], ['Career Progressio', 'Innovative Thinking', 'Skills & Talents '], ['Career Progression', 'Wor'], ['Career and Financ', 'Challenges & Setbacks ', 'Daily Routines ', 'Decluttering and Organization', 'Digital Footprint', 'Eco-Friendly Living ', 'Education', 'Goals & Aspirations', 'Happiness and Fulfillment ', 'Hobbies & Interests', 'Learning and Education', 'Lifelong Learning', 'Mental Wellness ', 'Mindful Living', 'Mindfulness and Meditation ', 'Minimalist Lifestyle', 'Minimizing & Evaluating Personal Belongings', 'Personal Development', 'Personality', 'Philosophical Beliefs ', 'Physical Health ', 'Relationships ', 'Self-Improvement', 'Spiritual Life', 'Work-Life Balance'], ['Community Engagement', 'Finance

In [26]:
import seaborn as sns
import random
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors

In [187]:
colors = colors2Community(communities)
colors.head(5)

node    color  group
0        Achievements   #57acdb      1
1       Personal Growt  #57acdb      1
2  Resilience and Grit  #57acdb      1
3    Anxiety and Fears  #dbae57      2
4    Coping Strategies  #dbae57      2

In [188]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [189]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show('keybert.html', notebook=False)

keybert.html


In [34]:
EXTRACT_KEYWORDS_PROMPT_PATH = "documents\master_prompts\keywords\extract_concepts.txt"
EXTRACT_KEYWORDS = {'path': EXTRACT_KEYWORDS_PROMPT_PATH, 'input_variables': ['conversation'], 'model_name': 'gpt-4o', 'temperature': 0.1}

CONCEPT2GRAPH_PROMPT_PATH = "documents\master_prompts\keywords\concept2graph.txt"
CONCEPT2GRAPH = {'path': CONCEPT2GRAPH_PROMPT_PATH, 'input_variables': ['input'], 'model_name': 'gpt-4o', 'temperature': 0.1}

In [35]:
# get conversations
conversations = eve.user_manager.database.get_session_history_data_as_dataframe(session_id=None)
conversations.head(1)

message_id message  session_id feedback  role  depression_score  \
0           1  Hello?           1     None  User          0.385345   

   stress_score  admiration  amusement     anger  ...      love  nervousness  \
0      0.385345     0.00165   0.001937  0.003768  ...  0.001283     0.000473   

    neutral  optimism     pride  realization    relief   remorse  sadness  \
0  0.931964  0.001528  0.000171     0.004223  0.000222  0.000259   0.0012   

   surprise  
0  0.004019  

[1 rows x 35 columns]

In [36]:
# from df to str in user: user message \n bot: bot message \n
def df2str(df):
    session_ids = df['session_id'].unique()
    conversations = []
    for session_id in session_ids:
        df_aux = df[df['session_id'] == session_id]

        conversations.append("\n".join(
        [f"user: {row['message']}" if row['role']=='User' else f"bot: {row['message']}" for _, row in df_aux.iterrows()]
        ))
    
    return conversations
conversation = df2str(conversations)

In [37]:
conversation

["user: Hello?\nbot: Hi Farzad, how are you? I'm sorry I didn't respond earlier. How can I help you today?\nuser: I'm at work today, not feeling well at all, I have some issues with management and some issues with my colleagues. I've tried to bring this up to the management, but nobody's actually listening to me. Thank you. \nbot: It sounds like you're feeling really frustrated and dissatisfied with work. I'm sorry to hear that you're not feeling heard by management. Can you tell me more about what's going on?\nuser: hello\nbot: Hello! How are you?\nuser: not good \nbot: I'm sorry to hear that. Can you tell me what's wrong?\nuser: im feeling really down \nbot: I'm sorry to hear that. Can you tell me what's wrong?\nuser: im not happy at work\nbot: I'm sorry to hear that. Can you tell me more about why you're not happy at work?\nuser: i feel I'm not being appreciated. \nbot: I'm sorry to hear that. It sounds like you're feeling really down and unhappy. Can you tell me more about what's g

In [38]:
mm = ModelsManager(prompts={'extract_keywords': EXTRACT_KEYWORDS, 'concept2graph': CONCEPT2GRAPH})

In [39]:
kw = []
for conv in conversation:
    mm.predict('extract_keywords', {'conversation':conv})
    kw.append(mm.information['extract_keywords'])

In [40]:
import json
import numpy as np

In [41]:
json_list = []
for k in kw:
    json_list.extend(json.loads(k))
#json_list = json.loads(kw)

In [42]:
def concepts2Df(concepts_list) -> pd.DataFrame:
    ## Remove all NaN entities
    concepts_dataframe = pd.DataFrame(concepts_list).replace(" ", np.nan)
    concepts_dataframe = concepts_dataframe.dropna(subset=["entity"])
    concepts_dataframe["entity"] = concepts_dataframe["entity"].apply(
        lambda x: x.lower()
    )

    return concepts_dataframe

In [43]:
cdf = concepts2Df(json_list)

In [44]:
mm.predict('concept2graph', {'input':json_list})
graph = mm.information['concept2graph']

In [45]:
graph_list = json.loads(graph)
graph_df = pd.DataFrame(graph_list)
graph_df

node_1                 node_2  \
0                    work             management   
1                    work             colleagues   
2                    work           feeling down   
3                    work  not being appreciated   
4                    work             overworked   
5                    work          too much work   
6                    work         not enough pay   
7                    work                 stress   
8                    work            overwhelmed   
9                    work             undermined   
10             management             colleagues   
11             management             HR manager   
12             colleagues                support   
13             colleagues                  ideas   
14             colleagues                  tasks   
15             colleagues       responsibilities   
16           feeling down          mental issues   
17           feeling down       emotional issues   
18  not being appreciated       emotional issues   
19             overworked                 stress   
20          too much work            overwhelmed   
21         not enough pay             undermined   
22         private issues                divorce   
23         private issues                   kids   
24               training             counseling   
25               training     therapy techniques   
26             counseling     therapy techniques   
27                support             counseling   
28                support     therapy techniques   

                                                 edge  
0   Management is an organizational aspect related...  
1   Colleagues are people associated with the plac...  
2   Feeling down is a condition that can occur at ...  
3   Not being appreciated is a condition that can ...  
4   Overworked is a condition that can occur at th...  
5   Too much work is a condition that can occur at...  
6   Not enough pay is a condition that can occur a...  
7   Stress is a condition that can occur at the pl...  
8   Overwhelmed is a condition that can occur at t...  
9   Undermined is a condition that can occur at th...  
10  Management interacts with colleagues in an org...  
11  HR manager is a part of management in an organ...  
12      Colleagues can provide support to each other.  
13        Colleagues can share ideas with each other.  
14               Colleagues can collaborate on tasks.  
15             Colleagues can share responsibilities.  
16      Feeling down can be related to mental issues.  
17   Feeling down can be related to emotional issues.  
18  Not being appreciated can lead to emotional is...  
19               Being overworked can lead to stress.  
20  Having too much work can make someone feel ove...  
21   Not enough pay can make someone feel undermined.  
22                Divorce is a type of private issue.  
23  Issues related to kids can be considered priva...  
24        Training can include counseling techniques.  
25           Training can include therapy techniques.  
26  Counseling can involve various therapy techniq...  
27        Support can be provided through counseling.  
28  Support can be provided through therapy techni...

In [46]:
nodes2 = pd.concat([graph_df['node_1'], graph_df['node_2']], axis=0).unique()
nodes2.shape

(24,)

In [47]:
## Add nodes to the graph
G2 = nx.Graph()
for node in nodes2:
    G2.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in graph_df.iterrows():
    G2.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
    )

In [48]:
communities_generator2 = nx.community.girvan_newman(G2)
top_level_communities2 = next(communities_generator2)
next_level_communities2 = next(communities_generator2)
communities2 = sorted(map(sorted, next_level_communities2))
print("Number of Communities = ", len(communities2))
print(communities2)

Number of Communities =  4
[['HR manager', 'colleagues', 'ideas', 'management', 'responsibilities', 'tasks'], ['counseling', 'support', 'therapy techniques', 'training'], ['divorce', 'kids', 'private issues'], ['emotional issues', 'feeling down', 'mental issues', 'not being appreciated', 'not enough pay', 'overwhelmed', 'overworked', 'stress', 'too much work', 'undermined', 'work']]


In [49]:
palette = "hls"

colors2 = colors2Community(communities2)
colors2.head(5)

node    color  group
0        HR manager  #91db57      1
1        colleagues  #91db57      1
2             ideas  #91db57      1
3        management  #91db57      1
4  responsibilities  #91db57      1

In [50]:
for index, row in colors2.iterrows():
    G2.nodes[row['node']]['group'] = row['group']
    G2.nodes[row['node']]['color'] = row['color']
    G2.nodes[row['node']]['size'] = G2.degree[row['node']]

In [51]:

net2 = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net2.from_nx(G2)
# net.repulsion(node_distance=150, spring_length=400)
net2.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net2.show_buttons(filter_=["physics"])

net2.show('gi.html', notebook=False)

gi.html


In [58]:
# get user dasboard
user_dashboard = eve.user_manager.database.get_user_dashboard(last_session=True)

In [59]:
user_dashboard

{'main_indicators': {'stress_score': 0.63,
  'delta_stress_score': 0.17,
  'depression_score': 0.62,
  'delta_depression_score': 0.43,
  'positive_score': 0.06,
  'delta_positive_score': 0.0,
  'neutral_score': 0.42,
  'delta_neutral_score': -0.16,
  'negative_score': 0.52,
  'delta_negative_score': 0.16},
 'positive_emotions': {'1_emotion_name': 'approval',
  '1_emotion_score': 2.18,
  '2_emotion_name': 'gratitude',
  '2_emotion_score': 2.13,
  '3_emotion_name': 'caring',
  '3_emotion_score': 1.62},
 'negative_emotions': {'1_emotion_name': 'disappointment',
  '1_emotion_score': 28.67,
  '2_emotion_name': 'annoyance',
  '2_emotion_score': 15.95,
  '3_emotion_name': 'sadness',
  '3_emotion_score': 10.09},
 'neutral_emotions': {'1_emotion_name': 'curiosity',
  '1_emotion_score': 15.44,
  '2_emotion_name': 'confusion',
  '2_emotion_score': 3.71,
  '3_emotion_name': 'realization',
  '3_emotion_score': 1.78},
 'mental_health': {0: {'period': '22/04/2024 - 28/04/2024',
   'positive_score': 0

In [60]:
# get user sessions
user_sessions = eve.user_manager.database.get_session_data_as_dataframe(session_id=None)

In [61]:
user_sessions

session_id                                            summary        date  \
0           1  Main topic of the conversation: Feeling unappr...  2024_04_22   
1           2  Main topic of the conversation: Feeling underm...  2024_04_26   

       duration  depression_score  stress_score  admiration  amusement  \
0  10180.505004          0.185024      0.453534    0.002698   0.001953   
1  10304.449604          0.833318      0.713992    0.002992   0.001386   

      anger  annoyance  ...     love  nervousness   neutral  optimism  \
0  0.008212   0.066277  ...  0.00221     0.003663  0.358773  0.003236   
1  0.012555   0.178220  ...  0.00215     0.035807  0.276394  0.005623   

      pride  realization    relief   remorse   sadness  surprise  
0  0.000414     0.009744  0.001693  0.006171  0.114502  0.002725  
1  0.000784     0.018615  0.003866  0.005117  0.074477  0.002193  

[2 rows x 34 columns]